In [2]:
import torch
import nltk
import re
import pandas as pd
import numpy
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, BertConfig
import preprocessor

In [3]:
from torch.utils.data import Dataset

In [4]:
#TODO Validation dataset from train
# separate DS for test
train = pd.read_csv('train.csv', encoding='utf-8')
test = pd.read_csv('test.csv', encoding='utf-8')

In [5]:
# exploring the data

In [6]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train.location.unique()

array([nan, 'Birmingham', 'Est. September 2012 - Bristol', ...,
       'Vancouver, Canada', 'London ', 'Lincoln'], dtype=object)

In [8]:
# train.keyword.unique()

In [9]:
# drop unnecessary columns which BERT won't need
# train.drop(['keyword', 'location'], axis=1)
# test.drop(['keyword', 'location'], axis=1)

In [10]:
# preprocess and tokenize the tweets
def preprocess_tweets(df):
    # lowercase
    df['text'] = df['text'].apply(lambda x: x.lower())

    # options - leave hashtags
    preprocessor.set_options(preprocessor.OPT.URL,
                             preprocessor.OPT.EMOJI,
                             preprocessor.OPT.SMILEY,
                             preprocessor.OPT.MENTION,
                             preprocessor.OPT.NUMBER)

    # replace numbers, as preprocessor module does not work with comma-separated numbers
    pattern = re.compile(r"(^|\s)(-?\d+([.,]?\d+))")
    df['text'] = df['text'].apply(lambda x: re.sub(pattern, "", x))
    
    # apply preprocessor module to remove redundant information
    df['text'] = df['text'].apply(preprocessor.clean)
    return df

In [11]:
train = preprocess_tweets(train)
test = preprocess_tweets(test)

In [12]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this #earthquake m...,1
1,4,NaN,NaN,forest fire near la ronge sask. canada,1
2,5,NaN,NaN,all residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,people receive #wildfires evacuation orders in...,1
4,7,NaN,NaN,just got sent this photo from ruby #alaska as ...,1


In [13]:
X_train = train['text']
X_test = test['text']
y_train = train['target']

In [14]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()

y_train = pd.get_dummies(y_train).values.tolist()

## Loading BERT from huggingface

Load tokenizer and model from huggingface

In [13]:
# tokenize words with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.save_pretrained("../bert-data-transformers")

In [ ]:
# train['text'] = train['text'].apply(tokenizer.tokenize)
# test['text'] = test['text'].apply(tokenizer.tokenize)

# train['text'] = train['text'].apply(tokenizer.encode)
# test['text'] = test['text'].apply(tokenizer.encode)

In [ ]:
# set model in training mode
# model.train()

In [25]:
max_seq_length = 128

class tweet_dataset(Dataset):
    def __init__(self,x_y_list):
        self.x_y_list = x_y_list
        
    def __getitem__(self,index):
        
        retweet = tokenizer.tokenize(self.x_y_list[0][index])
        
        if len(tokenized_tweet) > max_seq_length:
            tokenized_tweet = tokenized_tweet[:max_seq_length]
            
        ids_tweet  = tokenizer.convert_tokens_to_ids(tokenized_tweet)

        padding = [0] * (max_seq_length - len(ids_review))
        
        ids_tweet += padding
        
        assert len(ids_tweet) == max_seq_length
        
        #print(ids_review)
        ids_tweet = torch.tensor(ids_tweet)
        
        sentiment = self.x_y_list[1][index] # color        
        list_of_labels = [torch.from_numpy(np.array(sentiment))]
        
        
        return ids_tweet, list_of_labels[0]
    
    def __len__(self):
        return len(self.x_y_list[0])

In [1]:
train_lists = [X_train, y_train]
test_lists = [X_test, y_test]

training_dataset = text_dataset(x_y_list = train_lists )

test_dataset = text_dataset(x_y_list = test_lists )

dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

NameError: name 'X_train' is not defined

In [ ]:
train['text'] = train['text'].apply(tokenizer.tokenize)
test['text'] = test['text'].apply(tokenizer.tokenize)

train['text'] = train['text'].apply(tokenizer.encode)
test['text'] = test['text'].apply(tokenizer.encode)

In [ ]:
train.head()